In [14]:
import asyncio
import time
import logging
import threading
from pydantic import BaseModel, Field
from langchain.tools import tool
from controls import VoiceControl, WheelControlVibration

voice = VoiceControl()

# configure logging once at the start of your program
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s"
)


class VoiceAlertSchema(BaseModel):
    text: str = Field(..., description="The text to convert to speech.")


class VibrateSteeringSchema(BaseModel):
    duration: float = Field(..., gt=0, le=3, description="Duration in seconds")
    intensity: int = Field(..., ge=0, le=60, description="Vibration intensity")


def voice_alert(text: str) -> str:
    """Alert the driver via text-to-voice conversion."""

    def _speak():
        try:
            print("initialize voice alert")
            voice.text_to_speech(text)
            print("end=")
            return f"Voice alert started for text: '{text}'"
        except Exception as e:
            logging.error(f"[Voice Alert Error] {e}")

    threading.Thread(target=_speak, daemon=True).start()


# ---------------- Vibrate Steering ----------------
@tool(args_schema=VibrateSteeringSchema)
def vibrate_steering_wheel(duration: float, intensity: int) -> str:
    """Vibrate the steering wheel asynchronously."""

    def _vibrate():
        try:
            print("initialize vibrate steering wheel")
            steering = WheelControlVibration()
            print("end=initialize vibrate steering wheel")
            steering.vibrate(duration=duration, intensity=intensity)
            time.sleep(duration)
            steering.vibrate(duration=0)  # stop vibration
            print("end=vibrate steering wheel")
            return f"Steering wheel vibration started with intensity {intensity} for {duration} seconds."
        except Exception as e:
            logging.error(f"[Steering Wheel Error] {e}")

    threading.Thread(target=_vibrate, daemon=True).start()

In [ ]:
from langchain_core.messages import AIMessage
from langgraph.prebuilt import ToolNode

tool_node = ToolNode([vibrate_steering_wheel, voice_alert])

message_with_multiple_tool_calls = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "vibrate_steering_wheel",
            "args": {"duration": 2, "intensity": 20},
            "id": "9f2e4315-7e59-472b-837c-d548dc070209",
            "type": "tool_call",
        },
        {
            "name": "voice_alert",
            "args": {"text": "Alert: Driver is drowsy"},
            "id": "0bdca005-1bed-4912-8508-70db82c685f0",
            "type": "tool_call",
        },
    ],
)

In [16]:
tool_node.invoke({"messages": [message_with_multiple_tool_calls]})

initialize vibrate steering wheel
Failed to initialize Logitech G29 device logitech_raw. Vibration will not work.  Error: [Errno 2] No such file or directory: '/dev/logitech_raw'
end=initialize vibrate steering wheel
Raw device not initialized.  Cannot vibrate.
initialize voice alert
end=


{'messages': [ToolMessage(content='null', name='vibrate_steering_wheel', tool_call_id='9f2e4315-7e59-472b-837c-d548dc070209'),
  ToolMessage(content='null', name='voice_alert', tool_call_id='0bdca005-1bed-4912-8508-70db82c685f0')]}

Raw device not initialized.  Cannot vibrate.
end=vibrate steering wheel


In [18]:
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import ConfigurableField
from langchain_core.tools import tool


@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' times 'y'."""
    return x * y


@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the 'y'."""
    return x**y


@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y


tools = [multiply, exponentiate, add]

In [20]:
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, ToolMessage
from langchain_core.runnables import RunnableLambda
from langgraph.graph import END, StateGraph
from langchain_ollama import ChatOllama
from tools import (
    voice_alert,
    vibrate_steering_wheel,
)  # Assuming your tools are in this file
from pprint import pprint
import operator


# Define the state with the new pattern
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]


# Instantiate the tools and the model
tools = [vibrate_steering_wheel, voice_alert]
llm = ChatOllama(model="llama3.1:8b", temperature=0.0)
llm_with_tools = llm.bind_tools(tools)

In [21]:
import operator
from typing import Annotated, Sequence, TypedDict

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, ToolMessage
from langchain_core.runnables import RunnableLambda
from langgraph.graph import END, StateGraph


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]


def should_continue(state):
    return "continue" if state["messages"][-1].tool_calls else "end"


def call_model(state, config):
    return {"messages": [llm_with_tools.invoke(state["messages"], config=config)]}


def _invoke_tool(tool_call):
    tool = {tool.name: tool for tool in tools}[tool_call["name"]]
    return ToolMessage(tool.invoke(tool_call["args"]), tool_call_id=tool_call["id"])


tool_executor = RunnableLambda(_invoke_tool)


def call_tools(state):
    last_message = state["messages"][-1]
    return {"messages": tool_executor.batch(last_message.tool_calls)}


workflow = StateGraph(AgentState)
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tools)
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)
workflow.add_edge("action", "agent")
graph = workflow.compile()

In [23]:
graph.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "The driver is drowsy. Use the available tools "
                    "(voice_alert and vibrate_steering_wheel) to alert them. "
                    "Do not just give advice in text."
                ),
            }
        ]
    }
)

AttributeError: 'function' object has no attribute 'name'